### libs

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy import stats

In [10]:
import sklearn as sk

### data

In [ ]:
import seaborn as sns
df = sns.load_dataset('titanic')
print(f"{df.shape = }")
print(df.head().to_string())

df.shape = (891, 15)
   survived  pclass     sex   age  sibsp  parch     fare embarked  class    who  adult_male deck  embark_town alive  alone
0         0       3    male  22.0      1      0   7.2500        S  Third    man        True  NaN  Southampton    no  False
1         1       1  female  38.0      1      0  71.2833        C  First  woman       False    C    Cherbourg   yes  False
2         1       3  female  26.0      0      0   7.9250        S  Third  woman       False  NaN  Southampton   yes   True
3         1       1  female  35.0      1      0  53.1000        S  First  woman       False    C  Southampton   yes  False
4         0       3    male  35.0      0      0   8.0500        S  Third    man        True  NaN  Southampton    no   True


In [7]:
factors = [i for i in df.columns if i not in ['alive', 'survived']]
X = df[factors]
factors

['pclass',
 'sex',
 'age',
 'sibsp',
 'parch',
 'fare',
 'embarked',
 'class',
 'who',
 'adult_male',
 'deck',
 'embark_town',
 'alone']

In [8]:
y = df['survived']

### features

### model

In [11]:
# Example workflow for classification using sk alias
# 1. Load and prepare data
X_train, X_test, y_train, y_test = sk.model_selection.train_test_split(X, y, test_size=0.2, random_state=42)

# 2. Create and train model
model = sk.ensemble.GradientBoostingClassifier(random_state=42)
model.fit(X_train, y_train)

# 3. Make predictions and evaluate
y_pred = model.predict(X_test)
accuracy = sk.metrics.accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.4f}")
print(sk.metrics.classification_report(y_test, y_pred))

ValueError: could not convert string to float: 'male'

In [ ]:
# Example workflow for regression using sk alias
# 1. Prepare data
X_train, X_test, y_train, y_test = sk.model_selection.train_test_split(X, y, test_size=0.2, random_state=42)

# 2. Scale features if needed
scaler = sk.preprocessing.StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 3. Create and train model
model = sk.ensemble.GradientBoostingRegressor(random_state=42)
model.fit(X_train_scaled, y_train)

# 4. Evaluate
y_pred = model.predict(X_test_scaled)
mse = sk.metrics.mean_squared_error(y_test, y_pred)
r2 = sk.metrics.r2_score(y_test, y_pred)
print(f"MSE: {mse:.4f}")
print(f"R²: {r2:.4f}")

In [ ]:
# Other common models using sk alias
# Linear models
linear_reg = sk.linear_model.LinearRegression()
logistic_reg = sk.linear_model.LogisticRegression()

# Tree-based models
decision_tree = sk.tree.DecisionTreeClassifier()
random_forest = sk.ensemble.RandomForestClassifier()

# Preprocessing
label_encoder = sk.preprocessing.LabelEncoder()
standard_scaler = sk.preprocessing.StandardScaler()

### diagnostics and visualization

In [ ]:
# shapely
import shapely as sh

### misc

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin, is_outlier_detector
from sklearn.metrics import auc, mean_absolute_percentage_error, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, scale

x_train = 1
y_train = 1

train_test_split(x_train, y_train, test_size=0.2, random_state=0)

scaler = StandardScaler()
scaler.transform(x_train)
scaler.fit_transform(x_train)
scaler.scale_
scaler.mean_
